In [1]:
#import statements
import pandas as pd
import numpy as np
import funcs
%load_ext autoreload
%autoreload 2

In [2]:
#import climate data and covered storage area
#NOTE: kingferry and aurora have same climate data
castile_raw = pd.read_excel('../data/climate data.xlsx', sheet_name = 'castile')
kingferry_raw = pd.read_excel('../data/climate data.xlsx', sheet_name = 'kingferry')
aurora_raw = pd.read_excel('../data/climate data.xlsx', sheet_name = 'aurora')
fleming_raw = pd.read_excel('../data/climate data.xlsx', sheet_name = 'fleming')
leroy_raw = pd.read_excel('../data/climate data.xlsx', sheet_name = 'leroy')
castile_area = 118104 #ft^2
kingferry_area = 50400 + 77875 #ft^2, two storages, new + old
aurora_area = 108765 #ft^2
fleming_area = 130410 #ft^2
leroy_area = 58166 + 54900 #ft^2, two storages, new + old

#unit conversion using function in funcs.py
castile_raw= funcs.convert_units(castile_raw)
kingferry_raw= funcs.convert_units(kingferry_raw)
aurora_raw= funcs.convert_units(aurora_raw)
fleming_raw= funcs.convert_units(fleming_raw)
leroy_raw= funcs.convert_units(leroy_raw)

In [3]:
#mass balances and economic calculation using function in funcs.py
castile_mb = funcs.mass_bal(castile_raw, castile_area)
kingferry_mb = funcs.mass_bal(kingferry_raw, kingferry_area)
aurora_mb = funcs.mass_bal(aurora_raw, aurora_area)
fleming_mb = funcs.mass_bal(fleming_raw, fleming_area)
leroy_mb = funcs.mass_bal(leroy_raw, leroy_area)


In [4]:
#save mass balances in excel
with pd.ExcelWriter("../data/240415 mass balances, ijg.xlsx") as writer:
    castile_mb.to_excel(writer, sheet_name='castile')
    kingferry_mb.to_excel(writer, sheet_name='kingferry')
    aurora_mb.to_excel(writer, sheet_name='aurora')
    fleming_mb.to_excel(writer, sheet_name='fleming')
    leroy_mb.to_excel(writer, sheet_name='leroy')